In [10]:
%matplotlib inline

import sys
from urllib.request import urlretrieve
import zipfile
from dateutil.parser import parse
import json
from random import shuffle
import random
import datetime
import os

import boto3
import s3fs
import sagemaker
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import IntSlider, FloatSlider, Checkbox




In [11]:
region= sagemaker_session.boto_region_name
s3_data_path = "s3://jupstats"
freq = '1H'
# we predict for 0.5 days
prediction_length = 1 * 12
# we also use 0.5 days as context length, this is the number of state updates accomplished before making predictions
context_length = 1 * 12

estimator = sagemaker.estimator.Estimator(
    sagemaker_session=sagemaker.Session(),
    image_name=sagemaker.amazon.amazon_estimator.get_image_uri(region, "forecasting-deepar", "latest"),
    role=sagemaker.get_execution_role(),
    train_instance_count=1,
    train_instance_type='ml.c4.2xlarge',
    base_job_name='vsat',
    output_path="s3://jupstats/output"
)

hyperparameters = {
    "time_freq": freq,
    "epochs": "3",
    "early_stopping_patience": "40",
    "mini_batch_size": "64",
    "learning_rate": "5E-4",
    "context_length": str(context_length),
    "prediction_length": str(prediction_length)
}

estimator.set_hyperparameters(**hyperparameters)

In [12]:
%%time
data_channels = {
    "train": "{}/train/".format(s3_data_path),
    "test": "{}/test/".format(s3_data_path)
}

estimator.fit(inputs=data_channels, wait=True)

INFO:sagemaker:Creating training-job with name: vsat-2019-03-20-22-28-43-701


2019-03-20 22:28:43 Starting - Starting the training job...
2019-03-20 22:28:45 Starting - Launching requested ML instances.........
2019-03-20 22:30:20 Starting - Preparing the instances for training......
2019-03-20 22:31:43 Downloading - Downloading input data
2019-03-20 22:31:43 Training - Training image download completed. Training in progress..
Arguments: train
[03/20/2019 22:31:46 INFO 139955488274240] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/default-input.json: {u'num_dynamic_feat': u'auto', u'dropout_rate': u'0.10', u'mini_batch_size': u'128', u'test_quantiles': u'[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]', u'_tuning_objective_metric': u'', u'_num_gpus': u'auto', u'num_eval_samples': u'100', u'learning_rate': u'0.001', u'num_cells': u'40', u'num_layers': u'2', u'embedding_dimension': u'10', u'_kvstore': u'auto', u'_num_kv_servers': u'auto', u'cardinality': u'auto', u'likelihood': u'student-t', u'early_stopping_patience': u''}
[03

ValueError: Error for Training job vsat-2019-03-20-22-28-43-701: Failed Reason: ClientError: Only empty time series found in the dataset!